## Assignment 1
### Group 14 

> Github repository: https://github.com/LivDreyer/CSS24.git

> History of commits:

> Contribution:

In [17]:
#Importations 
import requests 
from bs4 import BeautifulSoup as BS
import pandas as pd 
import numpy as np
import ast
import networkx as nx
import matplotlib
import requests
from datetime import datetime

## Part 1: Web-scrabing 

In this part, we have included keynote speakers, chairs, authors of parallel talks, authors of posters, the names of researchers from the programme committee of the conference, and the organizers of tutorials. 

#### Names from the program: Plenary Speakers, Keynote Speakers, Authors of Parallel Talks, Authors of Posters, and Chairs
#### Link: https://ic2s2-2023.org/program

In [2]:
#Get html to py

link = "https://ic2s2-2023.org/program"
r = requests.get(link)
soup = BS(r.content)

In [3]:
# Plenary, parallel talks, posters speaker
# Name of final list: ppp_names

#find programs from each day 
plen = soup.find_all("ul", {"class":"nav_list"})

#find all names from the day 
names_plen = []
names_plen_text = []
for i in range(len(plen)): 
    names_plen.append(plen[i].find_all("i"))

for i in range(len(names_plen)):
    for j in range(len(names_plen[i])):
        names_plen_text.append(names_plen[i][j].text)

plen_names_lst = [name for names in names_plen_text for name in names.split(', ')]

ppp_names = []
for i in range(len(plen_names_lst)): 
    ppp_names.append(plen_names_lst[i].title())


for name in ppp_names:
    if '\u201a' in name:
        ppp_names.remove(name)

#print(ppp_names)

In [4]:
# Keynote speakers 
# Name of final list: key_names

table = soup.find_all("table", {"class": "tutorials"})
table = table[0].find_all("td")

table_text = []
for i in range(len(table)):
    txt = table[i].text
    if "Keynote" in txt: 
        table_text.append(txt)

key_names = []
for i in table_text: 
    key_names.append(i[10:])


#print(key_names)

In [5]:
# Chairs
# Name of final list: chair_names


chair = soup.find_all("div", {"class":"col-9 col-12-medium"})

#find chairs
names_chairs = []
names_chairs_text = []
for i in range(len(chair)): 
    names_chairs.append(chair[i].find_all("h2"))

for i in range(len(names_chairs)):
    for j in range(len(names_chairs[i])):
        names_chairs_text.append(names_chairs[i][j].text)


chr = []

for i in names_chairs_text:
    if "Chair" in i: 
        chr.append(i)

chair_names = []
for i in chr: 
    idx = i.find("Chair:")
    chair_names.append(i[idx + len("Chair:"):].strip())
    
    
#print(chair_names)

#### Names from the programme committee 
#### Link: https://ic2s2-2023.org/program_committee

In [6]:
link_pc = "https://ic2s2-2023.org/program_committee"
r_pc = requests.get(link_pc)
soup_pc = BS(r_pc.content)

In [7]:
# Programme Committee 
# Name of final list: pcom_names 

pc = soup_pc.find_all("article", {"class":"box post"})

#find all names from the day 
pc_names = []
pc_text = []
for i in range(len(pc)): 
    pc_names.append(pc[i].find_all("li"))

for i in range(len(pc_names)):
    for j in range(len(pc_names[i])):
        pc_text.append(pc_names[i][j].text)


pcom_names = []
for i in range(len(pc_text)):
    lst = pc_text[i].split(" (")
    pcom_names.append(lst[0])

#print(pcom_names)

#### Names from the Tutorials
#### Link: https://ic2s2-2023.org/tutorials

In [8]:
link_t = "https://ic2s2-2023.org/tutorials"
r_t = requests.get(link_t)
soup_t = BS(r_t.content)

In [9]:
# Tutorials 
# Name of final list: tut_names_f

tut = soup_t.find_all("div", {"class": "col-5 col-12-medium"})

#find all names from the day 
tut_names = []
tut_text = []
for i in range(len(tut)): 
    tut_names.append(tut[i].find_all("li"))

for i in range(len(tut_names)):
    for j in range(len(tut_names[i])):
        tut_text.append(tut_names[i][j].text)

#print(tut_text)

tut_names_f = []
for i in range(len(tut_text)):
    lst = tut_text[i].split(",")
    tut_names_f.append(lst[0])

#print(tut_names_f)

### Collect all names in one final string and process 

In [10]:
# Collect all names in one list of strings 

all_names = chair_names + key_names + ppp_names + pcom_names + tut_names_f
#print(all_names)

#Capitalize every name 
all_names_final = []
for i in range(len(all_names)): 
    all_names_final.append(all_names[i].title())

all_names_final = list(set(all_names_final))


#Pandas DF 
names_DF = pd.DataFrame(all_names_final, columns=['Researcher Name'])
names_DF = names_DF.sort_values(by='Researcher Name')

#Split names into first name and last name, dropping middle names in all formats: "John Name Doe" or "John N. Doe"
names_DF['first_name'] = names_DF['Researcher Name'].str.split().str[0]
names_DF['last_name'] = names_DF['Researcher Name'].str.split().str[-1]

#Remove duplicates of names such as "John Name Doe" or "John N. Doe"
names_DF.drop_duplicates(subset=['first_name', 'last_name'], inplace=True)

#Save first column (names) to file
names_DF['Researcher Name'].to_csv('/Users/livdreyerjohansen/Desktop/CSS/Names_DF_Ass1.csv', index=False)

#Find number of researchers

print("Number of names including optional exercises: ",len(names_DF))

Number of names including optional exercises:  1620


### How many unique researchers did you get? 

As seen above, we got 1620 unique names of researchers when including the optional exercises. 

### Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices (answer in max 150 words).

When initially inspecting the webpage to understand how it was structured, and where we should look, we relied on visual cues signaling what html-code corresponded to the data we wanted. Working with retrieving names of researchers from the program, we structured the retrieval in three parts, as we recognized different patterns for each type of name. We repeated this process for the optional exercise. For each part we used the [BeautifulSoup Python package](https://pypi.org/project/beautifulsoup4/)'s .[find_all](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all) method to extract the wanted names, and lastly combined every list of names to one final list. For the final list, we capitalized each name, and then used the set() method to remove apparent duplicates. We then created a Pandas Dataframe, where we split every name into first and last name excluding the middle name. It was then possible to remove rows with identical first and last name, letting us create a csv.file with unique “Researcher Names”.

## Part 2: Ready Made vs Custom Made Data

*Exercise: Ready made data vs Custom made data In this exercise, I want to make sure you have understood they key points of my lecture and the reading.*

**What are pros and cons of the custom-made data used in Centola's experiment (the first study presented in the lecture) and the ready-made data used in Nicolaides's study (the second study presented in the lecture)? You can support your arguments based on the content of the lecture and the information you read in Chapter 2.3 of the book (answer in max 150 words).**

Centola’s experiment [1] allows the investigation of diffusion processes with no influence from confounding variables such as advertising or social reinforcement from outside of the experiment. Though generally desired it’s hard to apply the results directly on real-word settings. Despite a smaller sample size, Centola could repeat the experiment to reproduce tendencies of spread of information. A problem of volunteer bias could however possibly arise from how he recruited participants.
In Nicolaides’s study [2] the large sample size and the non-reactivity of users constitute an advantage when investigating contagion as it can mirror real-world conditions. However, as mentioned in “Bit by bit” 2.3.3 [3], non-reactivity does not ensure that the data directly reflects behavior. Users of the fitness-app is not forced to participate even though they have connections on the app, and to paraphrase from the book: “It’s not that I am not exercising, I am just not posting it on thefitness- app”. 


**How do you think these differences can influence the interpretation of the results in each study? (answer in max 150 words)**

Centola’s findings are based on an artificial environment that find significant results mapping the “essence” of behavioral contagion, however, the reactivity of participants could influence the interpretation of the results. Additionally, as mentioned, the participants were recruited from health-sites which could promote the behavior Centola finds considering his experiment is also concerned with health behavior. 

Depending on how Nicolaides performs the linear regression to examine how one user’s exercise might influence their connections’ exercise on the fitness-tracker app, it might not mirror the actual contagion happening. If people are active users who follow their friends, but do not post their exercise themselves, the results of the linear regression may reflect this more than the actual influence users have on their connections.


[1] Damon Centola ,The Spread of Behavior in an Online Social Network Experiment.Science329,1194-1197(2010).DOI:10.1126/science.1185231

[2] Aral S, Nicolaides C. Exercise contagion in a global social network. Nat Commun. 2017 Apr 18;8:14753. doi: 10.1038/ncomms14753.

[3] Salganik, Matthew J. 2017. Bit by Bit: Social Research in the Digital Age. Princeton, NJ: Princeton University Press. Open review edition.

## Part 3: Gathering Research Articles using the OpenAlex API

### Step 1: Retrieve data. 

**Starting with the authors you identified in Week 2, Exercise 2, use the OpenAlex API works endpoint to fetch the research articles they have authored.**

## Part 4: The Network of Computational Social Scientists

### Part 1: Network Construction

**Weighted Edgelist Creation:** Start with your dataframe of papers. Construct a weighted edgelist where each list element is a tuple containing three elements: the author ids of two collaborating authors and the total number of papers they've co-authored. Ensure each author pair is listed only once.

In [16]:
#Load dataframe 

df = pd.read_csv("/Users/livdreyerjohansen/Desktop/CSS/final_papers_dataset.csv", usecols = ["id","author_ids"])

df['author_ids'] = df['author_ids'].apply(ast.literal_eval) # convert string to list

In [12]:
#Create List of Collaborations between authors

collabs = []
for work_idx in range(len(df["id"])):
    work = df["id"][work_idx]
    collabs.extend([(a, b) for idx, a in enumerate(df["author_ids"][work_idx]) for b in df["author_ids"][work_idx][idx + 1:]])
    if work_idx == 100:
        break

**Graph Construction:**

- Use NetworkX to create an undirected Graph.
- Employ the add_weighted_edges_from function to populate the graph with the weighted edgelist from step 1, creating a weighted, undirected graph.

In [13]:
#Count the number of occurences of the Collaborations

authors_collabs = []
for authors in collabs:
    count = 0
    for j in range(len(collabs)):
        if authors == collabs[j]:
            count += 1
    authors_collabs.append((authors[0],authors[1],count))

In [14]:
#Create graph G

G = nx.Graph()
G.add_weighted_edges_from(authors_collabs)
G.edges(data=True)

G.nodes['https://openalex.org/A5075442335']

{}

In [15]:
#Get unique authors in the collaborations
authors = []
for i in range(len(collabs)):
    if collabs[i][0] not in authors:
        authors.append(collabs[i][0])

**Node Attributes:**

- For each node, add attributes for the author's *display name*, *country*, *citation count*, and the *year of their first publication* in Computational Social Science. The *display name* and *country* can be retrieved from your *authors* dataset. The *year of their first publication* and the *citation count* can be retrieved from the *papers* dataset.
- Save the network as a JSON file.

In [18]:
G = nx.Graph()
G.add_weighted_edges_from(authors_collabs)
G.edges(data=True)

for i in range(1):
    BASE_URL = "https://api.openalex.org/authors"
    SEARCH = f"/{authors[i]}"
    SELECT = "?select=id,display_name,works_api_url,cited_by_count,last_known_institutions"

    req = requests.get(BASE_URL + SEARCH + SELECT)

    result = req.json()
    country = result['last_known_institutions'][0]['country_code']
    name = result['display_name']
    cites = result['cited_by_count']
    id = result['id']
    req2 = requests.get(result["works_api_url"])

    authors_work = req2.json()
    date = datetime.strptime("3000-01-01", "%Y-%m-%d")
    # for j in range(len(authors_work['results'])):
    for j in range(len(authors_work['results'])):
        field = authors_work['results'][j]['primary_topic']['field']['display_name']
        if field == "Sociology" or "Psychology" or "Economics" or "Political Science":
            date_string = authors_work['results'][j]['created_date']
            if date > datetime.strptime(date_string, "%Y-%m-%d"):
                date = datetime.strptime(date_string, "%Y-%m-%d")
                
    year_firstpub = date.date().year
    author_attribute = {'display_name': name, 'country': country, 'citation_count': cites, 'first_publication' : year_firstpub}
    nx.set_node_attributes(G, {id : author_attribute})
    print(id)

https://openalex.org/A5075442335


In [5]:
#Code JSON

### Part 2: Preliminary Network Analysis 

**Network Metrics**

**1. What is the total number of nodes (authors) and links (collaborations) in the network?**

Utilizing [G.number_of_nodes()](https://networkx.org/documentation/stable/tutorial.html) and [G.number_of_edges()](https://networkx.org/documentation/stable/tutorial.html) where G is our network graph, we find a total of 332 nodes (authors) and 6291 links (collaborations) in the network. 

**2. Calculate the network's density (the ratio of actual links to the maximum possible number of links). Would you say that the network is sparse? Justify your answer.**

When calculating the maximum number of links possible in an undirected graph using the formula (n*(n-1))/2, where n denotes number of nodes, we find that the maximum number of links for this graph is 54946. As mentioned above, our graph has a total of 6291 links equal to our network having a density of 0.114. **Would you say that the network is sparse? Justify your answer.**

**3. Is the network fully connected (i.e., is there a direct or indirect path between every pair of nodes within the network), or is it disconnected?**

Using the length of [list(nx.connected_components(G))](https://networkx.org/documentation/stable/tutorial.html) we see that the graph can be partitioned into multiple connected components meaning the network is considered disconnected. 

**4. If the network is disconnected, how many connected components does it have? A connected component is defined as a subset of nodes within the network where a path exists between any pair of nodes in that subset.**

Using the above method, and given that the network is disconnected, we find that there are 4 connected components. 

**5. How many isolated nodes are there in your network? An isolated node is defined as a node with no connections to any other node in the network.**

Using the code in the cell below, we see that there are no isolated nodes in the graph as there are no nodes where [G.degree\[node\]](https://networkx.org/documentation/stable/tutorial.html) is equal to 0. 

In [19]:
isolated = 0
for i in range(len(authors)):
    if G.degree(authors[i]) == 0:
        isolated +=1
print("Number of isolated nodes:", isolated)

Number of isolated nodes: 0


**6. Discuss the results above on network density, and connectivity. Are your findings in line with what you expected? Why? (answer in max 150 words)**

Answer

**Degree Analysis:**

**Compute the average, median, mode, minimum, and maximum degree of the nodes. Perform the same analysis for node strength (weighted degree). What do these metrics tell us about the network? (answer in max 150 words)**

**Top Authors:**

**1. Identify the top 5 authors by degree. What role do these node play in the network?**

**2. Research these authors online. What areas do they specialize in? Do you think that their work aligns with the themes of Computational Social Science? If not, what could be possible reasons? (answer in max 150 words)**